In [ ]:
import pormake as pm
import numpy as np
import csv
from collections import defaultdict

db = pm.Database()
loc = pm.Locator()

## metal nodes
Fe_oct_a = pm.BuildingBlock(bb_file="pi-d_building_blocks/Fe_oct.xyz")
Fe_oct_b = Fe_oct_a.make_chiral_building_block()
Fe_oct = Fe_oct_a

## organic nodes
PcM_O = pm.BuildingBlock(bb_file="pi-d_building_blocks/PcM_O.xyz")

## edges
spacer = pm.BuildingBlock(bb_file="pi-d_building_blocks/spacer.xyz")
DHBQ = pm.BuildingBlock(bb_file="pi-d_building_blocks/DHBQ.xyz")
Cu_pln = pm.BuildingBlock(bb_file="pi-d_building_blocks/Cu_pln.xyz")
pseudo_edge = pm.BuildingBlock(bb_file="pi-d_building_blocks/pseudo_edge.xyz")

pm.log.disable_print()
pm.log.disable_file_print()


## Prepare topologies for screening

In [ ]:
def get_divisors(num):
    
    divisors = []
    
    for i in range(1,num+1):
        if num % i == 0:
            divisors.append(i)
            
    return divisors

In [ ]:
def expand_topo(topo):
    x = 1
    y = 1
    z = 1
    edges = topo.edge_indices
    neigh = topo.neighbor_list
    for i in edges:

        ## below set lists unique neighbors for the edges. If 1, this means expansion is required.
        if len(set([j.index for j in neigh[i]])) == 1:
            dist = [abs(d) for d in neigh[i][0].distance_vector]

            ## expand topology along the "major" edge direction to allow for alternating node sequence.
            if x == 1 and dist.index(max(dist)) == 0:
                x = 2
            if y == 1 and dist.index(max(dist)) == 1:
                y = 2
            if z == 1 and dist.index(max(dist)) == 2:
                z = 2

    return topo * (x, y, z)

In [ ]:
## prep topologies for patterned node assignment

topo_codes = []
file = open('cn34_pid_topology_list.csv')
csvreader = csv.reader(file)

for row in csvreader:
    for topo in row:
        topo_codes.append(topo)

topos = []

for topo in topo_codes:
    cur_topo = db.get_topo(topo)
    cur_topo = expand_topo(cur_topo)

    if cur_topo.n_nodes % 2 == 1:
        ## expand in one of the directions to ensure even number of nodes
        cur_topo = cur_topo * (1, 1, 2)
        
    topos.append(cur_topo)

In [ ]:
## check rmsd and topology scale here (check # nodes again since expansion took place)

rmsd_topos = []
node_limit = 80

for topo in topos:
    
    max_rmsd = 0 

    ## limit the number of nodes to obtain MOFs with reasonable unti cell sizes
    if topo.n_nodes > node_limit:
        continue
    
    for ls in topo.unique_local_structures:
        
        if len(ls.positions) == 3:
            cur_rmsd = loc.calculate_rmsd(ls, Fe_oct)
            if cur_rmsd > max_rmsd:
                max_rmsd = cur_rmsd

        if len(ls.positions) == 4:
            cur_rmsd = loc.calculate_rmsd(ls, PcM_O)
            if cur_rmsd > max_rmsd:
                max_rmsd = cur_rmsd
    
    if max_rmsd < 0.1:
        rmsd_topos.append(topo)

print(len(rmsd_topos))


In [ ]:
cn34_topos = []

for topo in rmsd_topos:
    
    if max(topo.unique_cn) == 3:
        continue

    elif min(topo.unique_cn) == 4:
        continue
        
    else:
        cn34_topos.append(topo)
    

print(len(cn34_topos))

## Perform screening (try enantionmer arrangements)

In [ ]:
## attempt generation!
test_builder = pm.Builder(planarity_enforcement=True, angle_threshold=30, check_tetrahedral=False)
successful_cases = []

ML_edge = spacer
MM_edge = DHBQ
LL_edge = Cu_pln

for topo in cn34_topos:

    ## full single enantiomer considered only once
    single_enantiomer = False
    
    for chiral_grouping in [1, 2, 3, 4, 6, 8, 12, 16, 81]:

        ## full single enantiomer considered only once    
        num_nodes = len(topo.node_indices)        
        if single_enantiomer:
            continue
        if chiral_grouping >= num_nodes:
            single_enantiomer = True
        
        for chiral_begin_flip in [True, False]:       

            current_node = {}
            current_edge = {}
        
            ## initialize nodes
            for i, node in enumerate(topo.unique_cn):
                if node == 4:
                    current_node[i] = PcM_O
                if node == 3:
                    current_node[i] = Fe_oct_a
        
            ## initialize edges
            for i, edge in enumerate(topo.unique_edge_types):
                current_edge[tuple(edge)] = spacer
        
            bbs = test_builder.make_bbs_by_type(topo, current_node, current_edge)
            calc_permutation = test_builder.extract_permutation(topo, current_node, current_edge)
            calc_permutation = {i : perm for i, perm in enumerate(calc_permutation)}
            
            ## treat cn=4 cases. we do not have metal-based exp. cn=4 bbs, so enforce PcM linker for all cn=4 nodes.
            ## NOTE: For now, we leave out cases where cn=3 linkers are also mixed in.
        
            chiral_counter = 0
            chiral_flip = chiral_begin_flip
           
            for node in topo.node_indices:
        
                if topo.cn[node] == 4:
                    bbs[node] = PcM_O
        
                elif topo.cn[node] == 3:
                    
                    chiral_counter += 1
                    
                    if chiral_flip:
                        bbs[node] = Fe_oct_b
                    else:
                        bbs[node] = Fe_oct_a
                        
                    if chiral_counter % chiral_grouping == 0:
                        chiral_flip = not chiral_flip
                
        
            ## iterate through edges and assign edges accordingly
            for edge in topo.edge_indices:
        
                node_a, node_b = topo.get_neighbor_indices(edge)
            
                ## metal-metal case
                if bbs[node_a].name == 'Fe_oct' and bbs[node_b].name == 'Fe_oct':
                    bbs[edge] = MM_edge
        
                ## metal-linker case
                elif bbs[node_a].name == 'Fe_oct' and bbs[node_b].name != 'Fe_oct':
        
                    bbs[edge] = ML_edge
                    ## take care of spacer 
                    calc_permutation[edge] = [1, 0]
        
                ## linker-metal case
                elif bbs[node_a].name != 'Fe_oct' and bbs[node_b].name == 'Fe_oct':
        
                    bbs[edge] = ML_edge
                    ## take care of spacer             
                    calc_permutation[edge] = [0, 1]
        
                ## linker-linker case
                elif bbs[node_a].name != 'Fe_oct' and bbs[node_b].name != 'Fe_oct':
                    bbs[edge] = LL_edge
        
                else:
                    print('something\'s wrong!')
    
            try:
                mof = test_builder.build(topo, bbs, calc_permutation)
                mof.view()
                successful_cases.append((topo.name, chiral_grouping, chiral_begin_flip))
                print(topo.name, chiral_grouping, chiral_begin_flip)
                
            except:
                continue
        


## Final topologies to be kept

In [ ]:
successful_cases = [('fjh', 4, False),
 ('fog', 3, True),
 ('iab', 4, False),
 ('jph', 8, False),
 ('lil', 4, True),
 ('pto', 4, True),
 ('stu', 8, False),
 ('sur', 2, True)]